## Deep Learning Approaches for RF-based detection & classification
Transfer learning with a fully connected layer

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

# import the torch packages
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

import torchvision.models as models

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# import custom functions
from helper_functions import *
from latency_helpers import *
from loading_functions import *

from Torch_Models import *

from file_paths import *

from nn_functions import runkfoldcv
# from torchsummary import summary

In [9]:
import importlib
import Torch_Models
importlib.reload(Torch_Models)
from Torch_Models import *

### Load Features

In [3]:
feat_folder = '../Features/'
feat_name = 'RAW' # or RAW
t_seg = 20
n_per_seg = 1024

output_name = 'drones'
feat_format = 'IMG'
which_dataset = 'dronedetect'

# dataset specific parameters
drrf_highlow = 'H'
drde_ints = ['WIFI','CLEAN','BLUE','BOTH']

if feat_name == 'RAW':
    print ('Loading RAW '+ which_dataset +' data')
    if which_dataset == 'dronerf':
        dataset = RFRawTorch(dronerf_raw_path, t_seg, output_name, which_dataset, 
                             downsamp=10000, interferences=None)
    elif which_dataset == 'dronedetect':
        dataset = RFRawTorch(dronedetect_raw_path, t_seg, output_name, which_dataset, 
                             downsamp=10000, interferences=drde_ints)
else:
    if which_dataset == 'dronerf':
        print('Loading DroneRF Dataset')
        dataset = DroneRFTorch(dronerf_feat_path, feat_name, t_seg, n_per_seg,
                           feat_format, output_name, drrf_highlow)
    elif which_dataset == 'dronedetect':
        print('Loading DroneDetect Dataset')
        dataset = DroneDetectTorch(dronedetect_feat_path, feat_name, t_seg, n_per_seg, feat_format,
                                        output_name, drde_ints)

Loading RAW dronedetect data


get file lengths: 100%|████████████████████████████████████████████████████████████████████████████████████████| 390/390 [00:00<00:00, 754858.59it/s]


In [4]:
np.linspace(0, 10, num=5)

array([ 0. ,  2.5,  5. ,  7.5, 10. ])

In [5]:
feat, label = dataset.__getitem__(100)
# plt.imshow(feat)
# plt.show()
print(feat.shape)

torch.Size([2, 10000])


In [6]:
label

6

## Run Model

In [10]:
num_classes = len(dataset.unique_labels)
which_model = '1dconv' # or 'resnet'
if which_model == 'vgg':
    Model = VGGFC(num_classes)
elif which_model == 'resnet':
    Model = ResNetFC(num_classes)
elif which_model == '1dconv':
    Model = RFUAVNet(num_classes)

In [ ]:
test_samp = dataset.__getitem__(40)[0]
test_samp = torch.unsqueeze(test_samp, 0)
Model(test_samp)

In [44]:
# Configuration options
k_folds = 5

batch_size = 128 # 128
learning_rate = 0.01
num_epochs = 1 # 0
momentum = 0.95
l2reg = 1e-4

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [45]:
trainedModel, res_acc, res_f1, res_runtime = runkfoldcv(Model, dataset, device, k_folds, batch_size, learning_rate, num_epochs, momentum, l2reg)

--------------------------------
FOLD 0
--------------------------------
Starting epoch 1
('MIN', 'MIN', 'PHA', 'MA1', 'AIR', 'PHA', 'AIR', 'PHA', 'DIS', 'DIS', 'MA1', 'DIS', 'AIR', 'INS', 'PHA', 'PHA', 'DIS', 'PHA', 'INS', 'PHA', 'MA1', 'PHA', 'AIR', 'MAV', 'MA1', 'PHA', 'DIS', 'AIR', 'MIN', 'MIN', 'AIR', 'PHA', 'MIN', 'DIS', 'MAV', 'MIN', 'AIR', 'MIN', 'MAV', 'MAV', 'AIR', 'MAV', 'MIN', 'PHA', 'MIN', 'INS', 'MIN', 'INS', 'PHA', 'INS', 'INS', 'MA1', 'AIR', 'MA1', 'DIS', 'DIS', 'DIS', 'AIR', 'DIS', 'MAV', 'MIN', 'DIS', 'AIR', 'AIR', 'MIN', 'MA1', 'AIR', 'MA1', 'AIR', 'MIN', 'AIR', 'AIR', 'MIN', 'AIR', 'AIR', 'AIR', 'MIN', 'MIN', 'AIR', 'MAV', 'MAV', 'MIN', 'MIN', 'MA1', 'MAV', 'MAV', 'MAV', 'MA1', 'PHA', 'MAV', 'MAV', 'AIR', 'MIN', 'INS', 'MA1', 'MAV', 'MA1', 'INS', 'MIN', 'MAV', 'MA1', 'INS', 'MIN', 'INS', 'AIR', 'MIN', 'AIR', 'MIN', 'MA1', 'PHA', 'MIN', 'MA1', 'MA1', 'PHA', 'MAV', 'PHA', 'INS', 'MIN', 'INS', 'MAV', 'MA1', 'PHA', 'MAV', 'DIS', 'MAV', 'MA1', 'AIR', 'MAV')
Unexpected ex

Traceback (most recent call last):
  File "/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1981274/4105186653.py", line 1, in <cell line: 1>
    trainedModel, res_acc, res_f1, res_runtime = runkfoldcv(Model, dataset, device, k_folds, batch_size, learning_rate, num_epochs, momentum, l2reg)
  File "/home/kzhou/main/RFClassification/nn_functions.py", line 72, in runkfoldcv
    targets= targets.type(torch.long)
AttributeError: 'tuple' object has no attribute 'type'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packa

--------------------------------------------------------------------------------------------------------------

### Load gamutRF Features

In [1]:
label_dirs= {
    'drone': ['data/gamutrf-birdseye-field-days/leesburg_field_day_2022_06_15/worker1/','data/gamutrf-birdseye-field-days/pdx_field_day_2022_05_26/worker1/gamutrf/'], 
    'wifi_2_4': ['data/gamutrf-pdx/07_21_2022/wifi_2_4/'], 
    'wifi_5': ['data/gamutrf-pdx/07_21_2022/wifi_5/']
}

sample_secs = 0.02
nfft = 512
batch_size = 8
num_workers = 19
num_epochs = 4
train_val_test_split = [0.75, 0.05, 0.20]
save_iter = 200
eval_iter = 10000

dataset = GamutRFDataset(label_dirs, sample_secs=sample_secs, nfft=nfft)

NameError: name 'GamutRFDataset' is not defined

## VGG16 Features

In [69]:
vgg16 = models.vgg16(pretrained=True)
# summary(vgg16, (3,224,224))

modules=list(vgg16.children())[:-1]
vggmodel=nn.Sequential(*modules)

for p in vggmodel.parameters():
    p.requires_grad = False

In [70]:
# test one input
d = dataset.__getitem__(11)
inarr = d[0]
inarr = torch.moveaxis(inarr, 2, 0)
inarr =inarr.unsqueeze(dim=0)
print(inarr.shape)
# print(inarr.shape)
# postmove = inarr[0]
# print(premove==postmove) # confirm it is the same channel

# WHEN using single channel array format
# inputr = inarr.repeat(1,3,1,1)
# inputr = inputr.to(device)
out = vggmodel(inarr)

# reshape the output
out.flatten()

torch.Size([1, 3, 224, 224])


tensor([0., 0., 0.,  ..., 0., 0., 0.])

## Resnet Features

In [71]:
## Transfer learning from Resnet50 & Apply Logistic Regression (Swinney paper)

# use pretrained resnet feature and just keep up to the last layer
resnet50 = models.resnet50(pretrained=True)
modules=list(resnet50.children())[:-2]
resnet50=nn.Sequential(*modules)
for p in resnet50.parameters():
    p.requires_grad = False

In [72]:
# test resnet
# input = torch.randn(1,1,30,300)
d = dataset.__getitem__(0)
inarr = d[0]

inarr = torch.moveaxis(inarr, 2, 0)
inarr = inarr.unsqueeze(dim=0)
print(inarr.shape)
# resnet50(inarr).shape

resnet50(inarr).shape

torch.Size([1, 3, 224, 224])


torch.Size([1, 2048, 7, 7])

### Generate Pretrained/Transfer Learning Features

In [73]:
ModelDict = {'vgg':vggmodel, 'resnet': resnet50}

which_model = 'vgg'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
ModelDict[which_model] = ModelDict[which_model].to(device) #set model to device

Feats = []
y_num = [] # numerical values for y
for n in tqdm(range(len(dataset))):
    d = dataset.__getitem__(n)
    inarr = d[0]
#     inputr = inarr.repeat(1,3,1,1)  # repeat to have 3 channels of the same info
    inputr = torch.moveaxis(inarr, 2, 0) # move axis to have channels come first
    inputr =inputr.unsqueeze(dim=0)
    inputr = inputr.to(device)
    out = ModelDict[which_model](inputr)
    
    Feats.append(out.cpu().numpy().flatten())
    y_num.append(np.array(d[1]))

Feats = np.array(Feats)
y_num = np.array(y_num)

# flatten the middle dimension
Feats = Feats.reshape(Feats.shape[0], Feats.shape[-1])
# invert labels back to categorical
# vgg_y_cat = dataset.le.inverse_transform(vgg_y.astype(np.int64))
y_cat = np.array([dataset.idx_to_class[i] for i in y_num])

100%|██████████████████████████████████████████████████████████████████████████████████████████| 36778/36778 [02:11<00:00, 279.85it/s]


In [40]:
# Save VGG features
# vgg_save = {'feats': vgg_feats, 'y_cat':vgg_y_cat, 'y':vgg_y}
# file_name = 'transfer_learning_feats/VggFeats_'+str(seg_len)+'_'+str(n_per_seg)+'correct_psd_img'
# np.save(file_name, vgg_save)

### Run Classifier (LR or kNN)

In [74]:
Xs_feat = Feats # which features to use for logit reg
y_cat = y_cat
print('Shape of Input Features: (n sample x n feats):', Xs_feat.shape)

Shape of Input Features: (n sample x n feats): (36778, 25088)


In [75]:
## KFOLD split

k_fold = 5
cv = KFold(n_splits=k_fold, random_state=1, shuffle=True)


In [ ]:
# Apply PCA with LR and kNN
sc_ls = []
f1_ls = []
tm_ls = []
which_clf = 'kNN'
for train_ix, test_ix in tqdm(cv.split(Xs_feat)):
    scaler = StandardScaler()
    Xtrainscale = scaler.fit_transform(Xs_feat[train_ix])
    
    pca = PCA(n_components=10000)
    pca.fit(Xs_feat[train_ix])
    Xtrainpca = pca.transform(Xtrainscale)
    
    if which_clf == 'lr':
        clf = LogisticRegression(max_iter=10000)
    else:
        clf = KNeighborsClassifier(n_neighbors=5)
    
    clf.fit(Xtrainpca, y_cat[train_ix])
    
    # test
    start = time.time()
    Xtestscale = scaler.transform(Xs_feat[test_ix])
    Xtestpca = pca.transform(Xtestscale)
    y_pred = clf.predict(Xtestpca)
    end = time.time()
    
    t_ave = (end-start)/Xtestpca.shape[0] # batch measure time
    tm_ls.append(t_ave)
    
    sc = accuracy_score(y_cat[test_ix], y_pred)
    sc_ls.append(sc)
    f1 = f1_score(y_cat[test_ix], y_pred, average='weighted')
    f1_ls.append(f1)

out_msg = 'Net+LR: average test acc: {:.3}, F1: {:.3}, Run-time: {:.3}ms'.format(np.mean(sc_ls), np.mean(f1_ls), np.mean(tm_ls)*1e3)
print(out_msg)  

1it [09:03, 543.57s/it]

In [55]:
tm_ls

[0.0005148707321816803,
 0.0005234912641847237,
 0.0005121750670130158,
 0.0005138173020750074,
 0.0005133497936745498]

In [62]:
# Directly apply LR - too many features too long

# model parameters
# Cs=list(map(lambda x:pow(10,x),range(-2,2,1)))
# print('Cs:', Cs)

# best_params_ls = []
# acc_ls = []
# f1_ls = []
# runt_ls = []

# parameters = {'C':Cs}

# for train_ix, test_ix in tqdm(cv.split(Xs_feat)):
    
#     # find the optimal hypber parameters
#     lr = LogisticRegression(solver='saga')
# #     clf = GridSearchCV(lr, parameters, n_jobs=1) # gridsearch cv
#     clf = LogisticRegression(C =1.0, max_iter=5000, class_weight = 'balanced',n_jobs=1) # fixed parameter
    
#     scaler = preprocessing.StandardScaler().fit(Xs_feat[train_ix])
#     X_train_scale = scaler.transform(Xs_feat[train_ix])
    
#     clf.fit(X_train_scale, y_cat[train_ix])
    
# #     print(clf.best_params_)
# #     best_params_ls.append(clf.best_params_)
    
#     # predict on the test data
#     X_test_scale = scaler.transform(Xs_feat[test_ix])
#     y_pred, runtimes = atomic_benchmark_estimator(clf, X_test_scale, output_type= '<U3', 
#                                                   verbose=False)
#     runt_ls.append(np.mean(runtimes))
    
#     acc = accuracy_score(y_cat[test_ix], y_pred)
#     f1 = f1_score(y_cat[test_ix], y_pred, average='weighted')
#     print('Accuracy: {:.3},\t F1: {:.3}'.format(acc,f1))
#     acc_ls.append(acc)
#     f1_ls.append(f1)
    
# out_msg = 'Net+LR: average test acc: {:.2}, F1: {:.2}, Run-time: {:.2}ms'.format(np.mean(acc_ls), np.mean(f1_ls), np.mean(runt_ls)*1e3)
# print(out_msg)

In [12]:

# neigh = KNeighborsClassifier(n_neighbors=3)

In [ ]:
## Fixed parameter kNN (LONG run time - too many features)
# k_fold = 5
# cv = KFold(n_splits=k_fold, random_state=10, shuffle=True)

# # Ns=list(range(2,100,20))
# # Ns = [100]
# parameters = {'n_neighbors':Ns}
# print('list of neighbours:', Ns)

# Xs_arr = Vgg_Feats
# y_arr = Vgg_y_cat

# best_params_ls = []
# score_ls = []
# f1_ls = []

# for train_ix, test_ix in cv.split(Xs_arr):
#     # scale data
#     scaler = preprocessing.StandardScaler().fit(Xs_arr[train_ix])
#     X_train_scale = scaler.transform(Xs_arr[train_ix])
    
#     # find the optimal hypber parameters
#     clf = KNeighborsClassifier(n_neighbors=5)
# #     clf = GridSearchCV(neigh, parameters, n_jobs=1)
#     clf.fit(X_train_scale, y_arr[train_ix])
# #     print(clf.best_parameters)
    
#     # predict on the test data
#     X_test_scale = scaler.transform(Xs_arr[test_ix])
# #     y_pred = clf.predict(X_test_scale)
#     y_pred, runtimes = atomic_benchmark_estimator(clf, X_test_scale, output_type= '<U3', 
#                                                   verbose=False)
#     acc = accuracy_score(y_arr[test_ix], y_pred)
#     f1 = f1_score(y_arr[test_ix], y_pred, average='weighted')
#     f1_ls.append(f1)
#     print('Accuracy: {:.3},\t F1: {:.3}'.format(acc,f1))
#     score_ls.append(acc)
    
# print('VGG feats+kNN K-fold average test score:', np.mean(score_ls))
# print('VGG feats+kNN K-fold average test F1:', np.mean(f1_ls))

list of neighbours: [100]


## Fully Connected Layer

In [ ]:
# import Torch models
from Torch_Models import *

### Kfold CV

In [24]:
# Configuration options
k_folds = 5

batch_size = 128 # 128
num_classes = 7
learning_rate = 0.01
num_epochs = 10 # 0
momentum = 0.95
l2reg = 1e-4

Model = VGGFC(num_classes)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [26]:
trainedModel, res_acc, res_f1, res_runtime = runkfoldcv(Model, dataset, device, k_folds, batch_size, learning_rate, num_epochs, momentum, l2reg)

--------------------------------
FOLD 0
--------------------------------
Starting epoch 1
    Loss after mini-batch    50: 0.21611
    Loss after mini-batch   100: 0.23081
    Loss after mini-batch   150: 0.22870
    Loss after mini-batch   200: 0.23318
Starting epoch 2
    Loss after mini-batch    50: 0.22025
    Loss after mini-batch   100: 0.21542
    Loss after mini-batch   150: 0.22238
    Loss after mini-batch   200: 0.22282
Starting epoch 3
    Loss after mini-batch    50: 0.21271
    Loss after mini-batch   100: 0.21916
    Loss after mini-batch   150: 0.21282
    Loss after mini-batch   200: 0.21872
Starting epoch 4
    Loss after mini-batch    50: 0.20875
    Loss after mini-batch   100: 0.21702
    Loss after mini-batch   150: 0.21105
    Loss after mini-batch   200: 0.20857
Starting epoch 5
    Loss after mini-batch    50: 0.20477
    Loss after mini-batch   100: 0.20615
    Loss after mini-batch   150: 0.21356
    Loss after mini-batch   200: 0.20486
Starting epoch 6
    L

    Loss after mini-batch   200: 0.15825
Starting epoch 2
    Loss after mini-batch    50: 0.14772
    Loss after mini-batch   100: 0.15783
    Loss after mini-batch   150: 0.15562
    Loss after mini-batch   200: 0.15370
Starting epoch 3
    Loss after mini-batch    50: 0.15013
    Loss after mini-batch   100: 0.15044
    Loss after mini-batch   150: 0.14894
    Loss after mini-batch   200: 0.15829
Starting epoch 4
    Loss after mini-batch    50: 0.14647
    Loss after mini-batch   100: 0.14365
    Loss after mini-batch   150: 0.15970
    Loss after mini-batch   200: 0.15232
Starting epoch 5
    Loss after mini-batch    50: 0.15028
    Loss after mini-batch   100: 0.14706
    Loss after mini-batch   150: 0.14865
    Loss after mini-batch   200: 0.14988
Starting epoch 6
    Loss after mini-batch    50: 0.14911
    Loss after mini-batch   100: 0.14035
    Loss after mini-batch   150: 0.14213
    Loss after mini-batch   200: 0.14785
Starting epoch 7
    Loss after mini-batch    50: 0.14

## Save model

In [29]:
# Save model
model_type = 'VGGFC'
model_name = model_type+'_'+str(feat_name)+'_'+str(n_per_seg)+'_'+str(seg_len)
model_path = '../saved_models/'
trainedModel = trainedModel.cpu()
torch.save(trainedModel, model_path+model_name)
print('Model saved as:', model_name)

Model saved as: VGGFC_PSD_1024_20


In [16]:
### Test reload
# m2 = torch.load(model_path+model_name)
# in2 = dataset.__getitem__(0)[0]
# out = m2(in2)
# _,pred = torch.max(out,1)